In [2]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
from collections import Counter
stop_words = set(stopwords.words('english')) 

In [13]:
files = glob.glob('/Users/miriamblumenthal/Desktop/Desktop/GoogleTake2/search+youtube_v4/*.csv')

In [45]:
df_2020 = df_2020_weekend_events = df_2020_weekday_events = df_jan = df_feb = df_march = df_april = df_may = df_june = df_2019 = df_jan_march = df_health = pd.DataFrame(columns=pd.read_csv(files[0]).columns)
for f in files:
    df = pd.read_csv(f)
    df_jan = df_jan.append(df[(df.years == 2020) & (df.months == 1)], ignore_index=True)
    df_jan_health = df_jan.append(df[(df.years == 2020) & (df.months == 1) & (df.all_categories == "Health")], ignore_index=True)
    df_feb = df_feb.append(df[(df.years == 2020) & (df.months == 2)], ignore_index=True)
    df_feb_health = df_feb.append(df[(df.years == 2020) & (df.months == 2) & (df.all_categories == "Health")], ignore_index=True)
    df_march = df_march.append(df[(df.years == 2020) & (df.months == 3)], ignore_index=True)
    df_march_health = df_march.append(df[(df.years == 2020) & (df.months == 3) & (df.all_categories == "Health")], ignore_index=True)
    df_april = df_april.append(df[(df.years == 2020) & (df.months == 4)], ignore_index=True)
    df_april_health = df_april.append(df[(df.years == 2020) & (df.months == 4) & (df.all_categories == "Health")], ignore_index=True)
    df_may = df_may.append(df[(df.years == 2020) & (df.months == 5)], ignore_index=True)
    df_may_health = df_may.append(df[(df.years == 2020) & (df.months == 5) & (df.all_categories == "Health")], ignore_index=True)
    df_june = df_june.append(df[(df.years == 2020) & (df.months == 6)], ignore_index=True)
    df_health = df_health.append(df[df.all_categories == "Health"], ignore_index=True)
df_health

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,ratings,action,dislikes,likes,...,days,hours,minutes,weekdays,link,titles,source,query,datetime,all_categories
0,28,28,28,28,848,848,NaN,Searched,NaN,NaN,...,2,21,17,3,NaN,NaN,search,dementia,2020-01-02 21:17:00,Health
1,83,83,83,83,793,793,NaN,Searched,NaN,NaN,...,7,3,39,1,NaN,NaN,search,can you submit writing supplement after deadline,2020-01-07 03:39:00,Health
2,84,84,84,84,792,792,NaN,Visited,NaN,NaN,...,7,3,39,1,NaN,NaN,search,Submitting the Writing Supplement,2020-01-07 03:39:00,Health
3,85,85,85,85,791,791,NaN,Searched,NaN,NaN,...,7,3,42,1,NaN,NaN,search,can you submit upenn supplements after the dea...,2020-01-07 03:42:00,Health
4,91,91,91,91,785,785,NaN,Searched,NaN,NaN,...,7,3,54,1,NaN,NaN,search,can u send your supplements after deadline u of r,2020-01-07 03:54:00,Health
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3946,4420,4420,4420,4420,488,145,NaN,Visited,NaN,NaN,...,20,14,23,2,NaN,NaN,search,"Genetic analysis an integrated approach (Book,...",2020-05-20 14:23:00,Health
3947,4421,4421,4421,4421,487,144,NaN,Visited,NaN,NaN,...,20,14,23,2,NaN,NaN,search,Genetic analysis : an integrated approach (Boo...,2020-05-20 14:23:00,Health
3948,4476,4476,4476,4476,433,90,NaN,Searched,NaN,NaN,...,21,18,44,3,NaN,NaN,search,you are studying several types of an e coli he...,2020-05-21 18:44:00,Health
3949,4533,4533,4533,4533,379,36,NaN,Visited,NaN,NaN,...,25,12,2,0,NaN,NaN,search,"Peter R. Mariuz, M.D. - University of Rocheste...",2020-05-25 12:02:00,Health


In [59]:
def tokenize(df_name):
    text = df_name['query'].tolist()
    map(word_tokenize, text)
    df_name['query'].apply(word_tokenize)
    df_name['query'].apply(word_tokenize).tolist()
    tagged_list = nltk.pos_tag_sents(df_name['query'].apply(word_tokenize).tolist())
#     print(tagged_list)

In [54]:
def get_nouns():
    noun_list = []
    for i in tagged_list:
        temp = [j[0] for j in i if j[1].startswith("NN" or "NNP" or "NNS" or "NNPS")]
        noun_list.append(temp)


    noun_list = [[x.lower() for x in sublst] for sublst in noun_list]

#     print(noun_list)

In [55]:
def get_adj():
    adj_list = []
    for i in tagged_list:
        temp = [j[0] for j in i if j[1].startswith("JJ" or "JJR" or "JJS")]
        adj_list.append(temp)

    adj_list = [[x.lower() for x in sublst] for sublst in adj_list]

#     print(adj_list)

In [56]:
def get_verbs():
    verb_list = []
    for i in tagged_list:
        temp = [j[0] for j in i if j[1].startswith("VB" or "VBD" or "VBG" or "VBN" or "VBP" or "VBZ")]
        verb_list.append(temp)

    verb_list = [[x.lower() for x in sublst] for sublst in verb_list]

#     print(verb_list)

In [57]:
def bigram(list_name): 
    string = ""
    for l in list_name:
        if len(l) > 1:
            for word in l:
                string = string + " " + word
            tokens = nltk.word_tokenize(string)  
            bigrm = nltk.bigrams(tokens) 
            print(*map('-'.join, bigrm), sep=', ')
        else:
            print(l)
        string = ""
    


In [60]:
tokenize(df_health)
get_nouns()
get_adj()
get_verbs()
bigram(noun_list)
bigram(adj_list)
bigram(verb_list)

[['dementia'], ['supplement', 'deadline'], ['writing', 'supplement'], ['supplements', 'deadline'], ['supplements', 'u', 'r'], ['supplements', 'u', 'r', 'app'], ['supplements', 'deadline', 'app'], ['supplements', 'email'], ['supplements', 'deadline', 'app'], ['vision', 'rwanda'], ['problems', 'minneapolis'], ['problems', 'minneapolis', 'st', 'paul', 'thrillist'], ['blister'], ['cold', 'sore', 'fever', 'blister', 'intermountain', 'healthcare'], ['device'], ['hepatitis', 'b'], ['hepatitis', 'b'], ['coronavirus', 'treatment'], ['treatment', 'coronavirus'], ['treatment', 'coronavirus'], ['coronavirus', 'drugs'], ['coronavirus', 'drugs'], ['coronavirus', 'outbreak', 'coronavirus', 'drugs', 'vaccines'], ['public', 'health', 'guidance', 'potential', 'covid-19', 'exposure', 'cdc'], ['blushing'], ['blushing'], ['cases', 'u.s.', '|', 'cdc'], ['cdc', 'coronavirus', 'map'], ['sign', 'coronavirus'], ['elbow', 'joint', 'itchy'], ['paresthesia', 'arms', 'avoid'], ['deaths'], ['meal', 'plans', 'u', 'r'

management-benefits, benefits-university, university-rochester
adolescents-use, use-media
everything-coronavirus
['ivc.library.rochester']
ligament-sprain
carepass-terms, terms-conditions, conditions-cvs, cvs-pharmacy
androgen-insensitivity, insensitivity-syndrome
['hyperplasia']
hyperplasia-diagnosis, diagnosis-treatment, treatment-mayo
men-multivitamin
multivitamin-softgels
multivitamin-review, review-softgels
now-foods, foods-adam™, adam™-superior, superior-men, men-multi, multi-softgels, softgels-vitacost
foods-adam, adam-superior, superior-men, men-multi, multi-softgels, softgels-iherb
['https']
opko-health
fangbook-device
['cirrhosis']
gonadotropin-hormone
viral-sti, sti-incidence, incidence-rate
scabies-infections
phase-menses
lyme-disease
['metabolism']
acids-list, list-essential, essential-benefits
['mucus']
rem-sleep
rem-sleep, sleep-disorder
rem-sleep, sleep-disruption
stress-dream, dream-sleep, sleep-pathways, pathways-brain, brain-cell, cell-death
['dent']
human-ncbi
['str

coronavirus-covid-19, covid-19-questions, questions-cdc
california-distancing
covid-disease
inflammatory-syndrome
https-amp=true
test-past, past-infection, infection-antibody, antibody-test, test-cdc
covid-antibody, antibody-test, test-aetna
covid-antibody
covid-antibody, antibody-test
lab-patients, patients-covid-19, covid-19-antibody, antibody-test, test-online
imaging-laboratory, laboratory-diagnostics, diagnostics-solutions, solutions-siemens
valvula-mitral
['health']
tratamiento-preventivo, preventivo-para, para-malaria
sophie-turner
['implants']
['amgen']
['amgen']
uñas-gruesas, gruesas-foot, foot-health, health-facts
fungal-nail, nail-infections, infections-symptoms, symptoms-diagnosis, diagnosis-a, a-doctor
toe-fungus, fungus-treatment
swab-acv, acv-toe, toe-fungus
i-feel
surgeries-botched, botched-cost, cost-list
septum-treatment
borderline-personality, personality-disorder, disorder-|, |-reachout, reachout-australia
['eyes']
cdc-coronavirus, coronavirus-travel
coronavirus-uni

throat-covid, covid-symptoms
discharge-thick
cold-covid
['clinic']
afc-urgent, urgent-care, care-fairfield, fairfield-urgent, urgent-care, care-center, center-walk, walk-clinic
grade-fever
blood-type, type-covid19
['headache']
morning-headaches, headaches-causes, causes-prevention, prevention-healthline
headache-–, –-health, health-essentials, essentials-cleveland
occipital-neuralgia, neuralgia-brainfacts
flu-mortality, mortality-rate
influenza-pandemic, pandemic-virus
flu-death, death-toll, toll-tells, tells-us, us-covid-19
swine-flu, flu-origin
origins-h1n1, h1n1-pandemic, pandemic-swine, swine-mexico
vagina-bo
sweaty-vagina, vagina-tips, tips-treatment, treatment-prevention, prevention-healthline
different-vaginal, vaginal-smells, smells-healthline
indicate-glasses
skull-pain
headache-ear, ear-signs, signs-treatments
['hospital']
mclean-hospital, hospital-|, |-psychiatric, psychiatric-affiliate, affiliate-harvard, harvard-medical, medical-school
['coronavirus']
sex-differences, diff

mcat-practice, practice-test
virginia-commonwealth, commonwealth-university, university-school
grades-health, health-professions, professions-university, university-rochester
amino-acids
summer-research, research-experience, experience-program, program-cancer, cancer-science
center-autism, autism-research, research-excellence, excellence-–, –-care, care-boston, boston-university
autism-summer, summer-research
internship-center, center-autism, autism-research, research-car, car-children
autism-summer, summer-research, research-internship
myths-premed, premed-research, research-opportunities, opportunities-|, |-medical, medical-school
faja-lab, lab-boston, boston-children, children-hospital
faja-laboratory, laboratory-|, |-boston, boston-children, children-hospital
martina-poletti, poletti-ph.d., ph.d.-university, university-rochester, rochester-medical, medical-center
ur-medicine, medicine-neuroscience, neuroscience-–, –-martina, martina-poletti, poletti-phd
cell-cycle
acanthosis-nigric

...

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['susceptible']
[]
[]
['spectra']
['spectra']
[]
[]
[]
[]
['medical']
[]
[]
[]
[]
[]
[]
[]
[]
[]
clinical-pregnant
[]
[]
[]
[]
[]
[]
[]
['medical']
['medical']
['neonatal']
['genetic']
[]
['neonatal']
pregnant-worried, worried-new
['pregnant']
[]
['pregnant']
['amniotic']
[]
['ill-informed']
['caesarean']
['cesarean']
[]
[]
['chromosomal']
[]
['viral']
important-high
[]
['top']
[]
[]
['excess']
['excess']
['sn2']
[]
['glucose']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['more']
[]
[]
[]
['salary']
[]
[]
['//www.washingtonpost.com/health/2020/03/04/coronavirus-flu-comparison/']
cold-scientific, scientific-american
['optimum']
['optimum']
[]
long-covid-19, covid-19-infectious
long-covid-19, covid-19-infectious
[]
[]
[]
[]
['alira']
[]
[]
[]
['pharmaceutical']
['top']
[]
['rid']
effective-rid
['|']
[]
['bloody']
['evidence-based']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['free']
[]
[]
[]
['direct']
[]
[]
[]
[]
['direct']
[]
[]
['safe']
[]
['medtronic']
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
scan-mri
scan-mri
clearer-scan
['scan']
[]
[]
[]
[]
[]
[]
[]
[]
['american']
['2020-2021']
[]
[]
[]
[]
['npr']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['white']
[]
[]
medical-2019-2020
[]
[]
['comprehensive']
['adhd']
['psychiatric']
[]
[]
[]
[]
[]
[]
['essential']
['essential']
[]
[]
skin-public
[]
['common']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['noncompetitive']
[]
medical-average, average-multiple
['shemmassian']
[]
[]
['complex']
['more']
[]
['nuclear']
[]
[]
[]
['cardiac']
[]
[]
['glycaemic']
[]
[]
omega-fatty, fatty-good
[]
['medical']
[]
[]
pandemic-last
['hyaluronic']
[]
[]
[]
[]
[]
['last']
[]
['mental']
[]
[]
[]
ct-scan
[]
['partial']
[]
[]
['overall']
['medical']
['medical']
[]
['med']
['medical']
[]
[]
medical-|
['major']
[]
['medical']
['psych']
['episodic']
[]
[]
[]
[]
['topical']
['hyaluronic']
['hyaluronic']
[]
['hyaluronic']
[]
[]
[]
[]
[]
[]
['medical']
['b-']
[]
[]
thrifty-maternal
[]
[]
['premed']
ecological-rural
visual-tactile
[]
[]
[]
[]
[]
[]

['are']
['says']
[]
[]
['uses']
[]
[]
[]
[]
[]
[]
[]
[]
['does']
['does']
['does']
[]
[]
[]
's-live
[]
['fix']
[]
needed-treat
[]
[]
[]
[]
[]
['cause']
[]
[]
holds-deal, deal-vaping
[]
['caused']
['is']
[]
[]
[]
[]
[]
[]
[]
[]
['cite']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
do-have
do-have
[]
[]
[]
[]
[]
[]
is-get, get-did, did-drink
['know']
[]
is-does
['is']
['screening']
[]
[]
[]
['is']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['seroquel']
is-assisted
is-assisted
['•']
['treat']
['is']
[]
[]
[]
is-having
[]
[]
[]
[]
[]
[]
[]
[]
[]
['is']
[]
[]
['bottom']
tracking-heal
['help']
[]
[]
[]
['makes']
['makes']
[]
['causes']
[]
[]
[]
[]
[]
[]
['is']
[]
[]
[]
['prevent']
[]
['causes']
['spotting']
[]
['is']
['is']
does-have, have-heavy
[]
[]
does-drinking, drinking-hot
[]
['is']
['is']
['is']
[]
['guided']
[]
[]
[]
does-have
does-have
[]
['guided']
do-hurt
['aching']
[]
[]
[]
['flour']
['flour']
[]
[]
[]
[]
[]
['know']
[]
[]
[]
[]
[]
[]
[]
['adam']
['adam']
[]
[]
[]
[]
['imaging']
[]
releasing-

[]
['ob']
['use']
[]
[]
[]
obgyn-fairfield
[]
[]
['taking']
['taking']
[]
[]
['treat']
[]
does-do
['does']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['average']
[]
[]
[]
[]
[]
[]
["'re"]
[]
[]
[]
['peeling']
['peeling']
['get']
[]
[]
[]
[]
[]
['seem']
need-try
[]
[]
[]
[]
['degree']
['is']
['is']
['using']
[]
[]
flashing-asl
needed-get
[]
needed-get
[]
[]
[]
[]
[]
[]
[]
[]
[]
i-induce, induce-vomiting
i-induce, induce-vomiting
['get']
[]
[]
standard-be
[]
['is']
['is']
do-make
[]
['get']
[]
['is']
[]
[]
[]
[]
[]
['do']
[]
[]
['are']
are-lifting
be-made
['oxidized']
[]
['does']
['does']
[]
[]
hurt-chew
[]
[]
[]
[]
['mary']
['help']
['rgh']
['rgh']
[]
[]
[]
[]
[]
['getting']
[]
[]
[]
[]
[]
[]
[]
['explained']
[]
does-glucose, glucose-become
[]
['enter']
enter-be
['enter']
[]
is-switched, switched-deaf
did-switched
[]
['explained']
[]
['is']
does-drinking, drinking-get, get-bloating
changing-reduce
[]
[]
['res']
[]
are-swollen, swollen-hurt
[]
[]
[]
['make']
['is']
[]
do-targe

is-affected
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['is']
[]
[]
[]
[]
[]
['associated']
[]
[]
[]
[]
heating-induced
[]
[]
[]
[]
['stimulating']
[]
['immunostimulating']
[]
[]
are-used
[]
[]
['used']
[]
['using']
[]
['used']
[]
[]
[]
[]
['pulsed']
[]
[]
[]
[]
[]
['assisted']
[]
ushered-revolutionized
[]
[]
[]
[]
does-lasik
do-know, know-have
[]
are-detected
are-detected, detected-measured
do-set
do-set
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
do-use, use-get
[]
[]
[]
[]
[]
[]
ranked-news
[]
[]
[]
is-cupping
[]
[]
[]
[]
[]
[]
[]
[]
[]
['|']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['worldwide']
[]
[]
['worldwide']
['worldwide']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['have']
[]
[]
have-opt
[]
[]
['ems']
[]
['speaking']
[]
[]
did-did, did-come
['stop']
[]
['is']
[]
['is']
['updates']
[]
[]
[]
encourages-dial
[]
do-become
become-is
['become']
want-be
['become']
want-be
['become']
[]
chose-become
want-be
become-is
['are']
[]
['are']
['are']
[]
['are']
physicians—and-according
['learned']
is-become
['choose']
[]
[]
[]
